In [ ]:
# Phân loại cảm xúc IMDB sử dụng BiLSTM + Attention

Trong notebook này, chúng ta sẽ xây dựng mô hình BiLSTM kết hợp với lớp Attention để phân loại cảm xúc (tích cực/tiêu cực) trên bộ dữ liệu IMDb movie reviews.

## Mục tiêu:
- Xây dựng mô hình BiLSTM + Attention từ đầu
- Huấn luyện trên dataset IMDb 
- Đánh giá hiệu quả mô hình
- So sánh với các phương pháp khác

## Các bước thực hiện:
1. Import thư viện và thiết lập môi trường
2. Load và tiền xử lý dữ liệu IMDb
3. Xây dựng lớp Attention tùy chỉnh
4. Tạo mô hình BiLSTM + Attention
5. Huấn luyện mô hình
6. Đánh giá và trực quan hóa kết quả
7. Test với dữ liệu mới

### 1. Import thư viện và thiết lập môi trường

Import các thư viện cần thiết cho việc xây dựng mô hình BiLSTM + Attention:

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Thiết lập seed cho reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

In [ ]:
### 2. Tạo lớp Attention tùy chỉnh

Xây dựng lớp Attention để tăng khả năng tập trung vào những phần quan trọng trong chuỗi văn bản:

In [ ]:
class AttentionLayer(layers.Layer):
    """
    Custom Attention Layer cho BiLSTM
    Cho phép mô hình tập trung vào các phần quan trọng trong chuỗi đầu vào
    """
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Ma trận trọng số attention
        self.W = self.add_weight(name="attention_weight", 
                                shape=(input_shape[-1], 1),
                                initializer='random_normal',
                                trainable=True)
        # Bias cho attention
        self.b = self.add_weight(name="attention_bias", 
                                shape=(input_shape[1], 1),
                                initializer='zeros',
                                trainable=True)        
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, x):
        # x shape: (batch_size, time_steps, features)
        # Tính attention scores
        e = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        # Chuẩn hóa attention weights bằng softmax
        a = tf.nn.softmax(e, axis=1)
        
        # Áp dụng attention weights lên input
        output = x * a
        
        # Tổng hợp thông tin từ tất cả time steps
        return tf.reduce_sum(output, axis=1)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

print("Lớp AttentionLayer đã được định nghĩa thành công!")

### 3. Load và tiền xử lý dữ liệu IMDb

Tải bộ dữ liệu IMDb movie reviews và thực hiện tiền xử lý:

In [ ]:
def load_and_preprocess_data(max_features=10000, maxlen=500):
    """
    Load và tiền xử lý dữ liệu IMDb
    """
    print("Đang tải dữ liệu IMDb...")
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    
    print(f"Số lượng reviews training: {len(x_train)}")
    print(f"Số lượng reviews testing: {len(x_test)}")
    print(f"Số lượng từ tối đa: {max_features}")
    print(f"Độ dài sequence tối đa: {maxlen}")
    
    # Kiểm tra phân bố nhãn
    print(f"\nPhân bố nhãn training:")
    print(f"- Negative (0): {sum(y_train == 0)} reviews")
    print(f"- Positive (1): {sum(y_train == 1)} reviews")
    
    # Padding sequences để có cùng độ dài
    print("\nĐang padding sequences...")
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)
    
    print(f"Shape sau padding:")
    print(f"x_train: {x_train.shape}")
    print(f"x_test: {x_test.shape}")
    
    return (x_train, y_train), (x_test, y_test)

# Thiết lập tham số
MAX_FEATURES = 10000  # Số từ vựng tối đa
MAXLEN = 500         # Độ dài sequence tối đa

# Load dữ liệu
(x_train, y_train), (x_test, y_test) = load_and_preprocess_data(MAX_FEATURES, MAXLEN)

### 4. Xây dựng mô hình BiLSTM + Attention

Tạo mô hình sử dụng lớp BiLSTM kết hợp với Attention mechanism:

In [ ]:
def create_bilstm_attention_model(max_features, maxlen, embedding_dim=128, lstm_units=64):
    """
    Tạo model BiLSTM với Attention
    """
    model = models.Sequential()
    
    # Lớp Embedding: chuyển từ số thành vector đặc trưng
    model.add(layers.Embedding(max_features, embedding_dim, input_length=maxlen))
    model.add(layers.Dropout(0.2))
    
    # Hai lớp BiLSTM để học đặc trưng từ cả hai hướng
    model.add(layers.Bidirectional(
        layers.LSTM(lstm_units, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)
    ))
    model.add(layers.Bidirectional(
        layers.LSTM(lstm_units, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)
    ))
    
    # Lớp Attention để tập trung vào phần quan trọng
    model.add(AttentionLayer())
    model.add(layers.Dropout(0.5))
    
    # Các lớp Dense để phân loại
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.3))
    
    # Lớp output: phân loại nhị phân (positive/negative)
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

# Tham số mô hình
EMBEDDING_DIM = 128  # Dimension của embedding
LSTM_UNITS = 64      # Số units trong LSTM

# Tạo mô hình
print("Đang xây dựng model...")
model = create_bilstm_attention_model(MAX_FEATURES, MAXLEN, EMBEDDING_DIM, LSTM_UNITS)

# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Hiển thị kiến trúc mô hình
print("\nKiến trúc mô hình:")
model.summary()

### 5. Huấn luyện mô hình

Thiết lập callbacks và bắt đầu quá trình huấn luyện:

In [ ]:
# Tham số huấn luyện
BATCH_SIZE = 32      # Batch size
EPOCHS = 10          # Số epochs

# Thiết lập callbacks để tối ưu hóa quá trình huấn luyện
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)
]

# Bắt đầu huấn luyện
print("Bắt đầu huấn luyện mô hình...")
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {EPOCHS}")
print(f"Callbacks: EarlyStopping, ReduceLROnPlateau")

history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(x_test, y_test),
    callbacks=callbacks,
    verbose=1
)

print("\nHuấn luyện hoàn tất!")

### 6. Trực quan hóa quá trình huấn luyện

Vẽ biểu đồ để quan sát quá trình huấn luyện:

In [ ]:
def plot_training_history(history):
    """
    Vẽ biểu đồ quá trình training
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training & validation accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
    ax1.set_title('Độ chính xác mô hình qua các epoch')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot training & validation loss
    ax2.plot(history.history['loss'], label='Training Loss', marker='o')
    ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    ax2.set_title('Loss mô hình qua các epoch')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Vẽ biểu đồ
plot_training_history(history)

In [ ]:
### 7. Đánh giá mô hình

Đánh giá hiệu quả mô hình trên tập test và hiển thị các metrics chi tiết:

def evaluate_model(model, x_test, y_test):
    """
    Đánh giá model và hiển thị kết quả chi tiết
    """
    # Dự đoán trên tập test
    print("Đang thực hiện dự đoán trên tập test...")
    y_pred_prob = model.predict(x_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Classification report
    print("\n" + "="*60)
    print("BÁO CÁO PHÂN LOẠI CHI TIẾT")
    print("="*60)
    print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Positive'], 
                yticklabels=['Negative', 'Positive'])
    plt.title('Ma trận nhầm lẫn (Confusion Matrix)')
    plt.xlabel('Dự đoán')
    plt.ylabel('Thực tế')
    plt.show()
    
    # Test accuracy
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"\n{'='*60}")
    print(f"KẾT QUẢ CUỐI CÙNG")
    print(f"{'='*60}")
    print(f"Độ chính xác trên tập test: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"Loss trên tập test: {test_loss:.4f}")
    
    return y_pred, y_pred_prob

# Đánh giá mô hình
y_pred, y_pred_prob = evaluate_model(model, x_test, y_test)

### 8. Test với dữ liệu mới

Thử nghiệm mô hình với một số reviews mẫu để xem khả năng dự đoán:

In [ ]:
def predict_sentiment(model, text, word_index, maxlen=500):
    """
    Dự đoán cảm xúc cho một text mới
    """
    # Chuyển text thành sequence
    sequence = []
    for word in text.lower().split():
        if word in word_index and word_index[word] < 10000:
            sequence.append(word_index[word])
    
    # Nếu sequence rỗng, return unknown
    if not sequence:
        print(f"Text: {text}")
        print("Không thể xử lý text này (không có từ nào trong từ điển)")
        print("-" * 50)
        return
    
    # Padding
    sequence = pad_sequences([sequence], maxlen=maxlen)
    
    # Predict
    prediction = model.predict(sequence, verbose=0)[0][0]
    
    sentiment = "Tích cực (Positive)" if prediction > 0.5 else "Tiêu cực (Negative)"
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    print(f"Text: {text}")
    print(f"Cảm xúc: {sentiment}")
    print(f"Độ tin cậy: {confidence:.4f} ({confidence*100:.2f}%)")
    print(f"Điểm số raw: {prediction:.4f}")
    print("-" * 50)

# Load word index để chuyển đổi text
word_index = imdb.get_word_index()

print("KIỂM THỬ MÔ HÌNH VỚI CÁC REVIEWS MẪU")
print("="*60)

# Test với các examples khác nhau
test_texts = [
    "This movie is absolutely fantastic! Great acting and amazing storyline.",
    "Terrible film. Boring plot and bad acting. Complete waste of time.",
    "The movie was okay, nothing special but not bad either.",
    "One of the best movies I have ever seen! Highly recommended!",
    "Awful movie. Poor direction and terrible script. Very disappointed.",
    "Amazing cinematography and outstanding performances from all actors.",
    "I fell asleep halfway through. Very boring and predictable plot.",
    "Perfect movie for the weekend. Really enjoyed watching it.",
    "Not my cup of tea but can understand why others might like it.",
    "Masterpiece! Every scene was perfectly crafted and emotionally engaging."
]

for text in test_texts:
    predict_sentiment(model, text, word_index, MAXLEN)

### 9. Lưu mô hình

Lưu lại mô hình đã huấn luyện để sử dụng sau này:

In [ ]:
# Lưu mô hình
model.save('bilstm_attention_imdb_model.h5')
print("✅ Mô hình đã được lưu thành 'bilstm_attention_imdb_model.h5'")

# Lưu thông tin về quá trình huấn luyện
import json

training_info = {
    'max_features': MAX_FEATURES,
    'maxlen': MAXLEN,
    'embedding_dim': EMBEDDING_DIM,
    'lstm_units': LSTM_UNITS,
    'batch_size': BATCH_SIZE,
    'epochs_trained': len(history.history['loss']),
    'final_train_accuracy': float(history.history['accuracy'][-1]),
    'final_val_accuracy': float(history.history['val_accuracy'][-1]),
    'final_train_loss': float(history.history['loss'][-1]),
    'final_val_loss': float(history.history['val_loss'][-1])
}

with open('training_info.json', 'w') as f:
    json.dump(training_info, f, indent=2)

print("✅ Thông tin huấn luyện đã được lưu vào 'training_info.json'")
print("\nThông tin mô hình:")
for key, value in training_info.items():
    if 'accuracy' in key or 'loss' in key:
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

### 10. Tóm tắt và so sánh

#### Ưu điểm của mô hình BiLSTM + Attention:
- **Hiểu ngữ cảnh hai chiều**: BiLSTM xử lý thông tin từ cả hai hướng (trước và sau)
- **Tập trung vào phần quan trọng**: Lớp Attention giúp mô hình chú ý đến những từ quan trọng nhất
- **Hiệu quả với dữ liệu tuần tự**: Phù hợp với đặc tính của văn bản là dữ liệu tuần tự
- **Tốc độ huấn luyện hợp lý**: Nhanh hơn so với các mô hình Transformer lớn

#### Nhược điểm:
- **Độ chính xác thấp hơn BERT**: Có thể không đạt được độ chính xác cao như các mô hình pretrained lớn
- **Cần nhiều dữ liệu**: Hiệu quả tốt nhất khi có lượng dữ liệu huấn luyện lớn
- **Khó xử lý mối quan hệ xa**: LSTM có thể gặp khó khăn với các phụ thuộc xa trong văn bản

#### So sánh với BERT:
- **BERT**: Độ chính xác cao hơn nhưng cần nhiều tài nguyên tính toán và thời gian huấn luyện
- **BiLSTM + Attention**: Cân bằng giữa hiệu quả và tốc độ, phù hợp khi tài nguyên hạn chế

Mô hình BiLSTM + Attention là một lựa chọn tốt khi cần cân bằng giữa độ chính xác và hiệu quả tính toán!